In [1]:
import os
import sys
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

from helper_functions import *
from analytic_functions import *

pd.set_option('max_colwidth', None)

In [2]:
total_messages, total_messages_with_id, participant_names = get_messages()
len(total_messages)

44595

In [3]:
participant = 'Rene Carballo'

### who has sent the most messages

In [4]:
d = message_count(total_messages, participant_names)

### most character count

In [5]:
d = average_character_count(total_messages, participant_names)

In [6]:
d =total_character_count(total_messages, participant_names)

### most reacted to message

In [7]:
most_reacted_to_message(total_messages_with_id)

{'sender_name': 'Kevin Tu',
 'timestamp_ms': 1594843341343,
 'content': 'Wow I just got an offer for full time at raytheon',
 'reactions': [{'reaction': 'ð\x9f\x92\x97', 'actor': 'Armando Amigon'},
  {'reaction': 'ð\x9f\x92\x97', 'actor': 'Feme Longe'},
  {'reaction': 'â\x9d¤', 'actor': 'Michael Sprintson'},
  {'reaction': 'â\x9d¤', 'actor': 'Zach Alvear'},
  {'reaction': 'â\x9d¤', 'actor': 'Anirudh Kuchibhatla'},
  {'reaction': 'ð\x9f\x92\x97', 'actor': 'Rene Carballo'},
  {'reaction': 'â\x9d¤', 'actor': 'Joshua Bae'},
  {'reaction': 'ð\x9f\x92\x97', 'actor': 'Patrick Chickey'},
  {'reaction': 'â\x9d¤', 'actor': 'Justin Cheung'},
  {'reaction': 'ð\x9f\x92\x97', 'actor': 'Fredy Martinez'},
  {'reaction': 'â\x9d¤', 'actor': 'Kevin Mejia'}],
 'type': 'Generic'}

### recieved reactions for every participant

In [8]:
render_mpl_table(reactions_recieved_by_all(participant_names, total_messages, returntype='pd').reset_index())
plt.savefig('pictures/reactions_recieved_by_all.png')
plt.close()

### specific react to messages reatio

In [9]:
react_stats = reactions_recieved_by_all(participant_names, total_messages, returntype='dict')

### reactions given to participant per person

In [10]:
render_mpl_table(reactions_recieved_by_participant(total_messages, participant).reset_index())
plt.savefig(f'pictures/reactions_recieved_by_participant {participant}')
plt.close()

### who has reacted what the most

In [11]:
render_mpl_table(reactions_given_by_all(total_messages).reset_index())
plt.savefig(f'pictures/reactions_given_by_all')
plt.close()

### reactions given by participant

In [12]:
df = reactions_given_by_participant(total_messages,participant)

In [13]:
render_mpl_table(df.reset_index())
plt.savefig(f'pictures/reactions_given_by_participant {participant}')
plt.close()

### average reactions to participant message and max reactions

In [14]:
average_and_max_reactions_for_participant(total_messages, participant)

(2.0, 2)

### which message did kevin react to himself

In [15]:
pd.DataFrame(self_reacts(total_messages, participant))

,sender_name,timestamp_ms,content,reactions,type
0,Rene Carballo,1569268770587,ð½,"[{'reaction': 'ð', 'actor': 'Zach Alvear'}, {'reaction': 'ð', 'actor': 'Anirudh Kuchibhatla'}, {'reaction': 'ð', 'actor': 'Rene Carballo'}]",Generic
1,Rene Carballo,1571085702589,I got 1/2,"[{'reaction': 'ð ', 'actor': 'Anirudh Kuchibhatla'}, {'reaction': 'ð', 'actor': 'Rene Carballo'}, {'reaction': 'ð', 'actor': 'Fredy Martinez'}, {'reaction': 'ð', 'actor': 'Kevin Tu'}]",Generic
2,Rene Carballo,1575479400153,Oh how far did you guys get in 303 yesterday?,"[{'reaction': 'ð', 'actor': 'Rene Carballo'}, {'reaction': 'ð', 'actor': 'Fredy Martinez'}, {'reaction': 'ð', 'actor': 'Kevin Mejia'}]",Generic
3,Rene Carballo,1586362084611,Is she still technically SA president?,"[{'reaction': 'ð', 'actor': 'Armando Amigon'}, {'reaction': 'ð', 'actor': 'Feme Longe'}, {'reaction': 'ð¢', 'actor': 'Rene Carballo'}]",Generic
4,Rene Carballo,1588539410457,Wait can I join if I'm not in the Spotify plan?,"[{'reaction': 'ð¢', 'actor': 'Rene Carballo'}, {'reaction': 'ð', 'actor': 'Kevin Mejia'}]",Generic
5,Rene Carballo,1593302357225,Labview is a p useful programming language,"[{'reaction': 'ð', 'actor': 'Rene Carballo'}, {'reaction': 'ð', 'actor': 'Justin Cheung'}]",Generic
6,Rene Carballo,1595449980510,Cd's nuts,"[{'reaction': 'ð ', 'actor': 'Armando Amigon'}, {'reaction': 'ð', 'actor': 'Rene Carballo'}]",Generic


### kevins most common words

In [16]:
plot_dict(most_common_words_by_participant(total_messages, participant).head(20).to_dict());
plt.savefig(f'pictures/most_common_words_{participant}.png')
plt.close()

### groups most common words

In [17]:
# most_common_words([i['content'] for i in total_messages if 'content' in i]).head(20)

### all nicknames of particpant

In [18]:
pnamereturn, ptimereturn = nickname_changes_and_times(total_messages, participant)

In [19]:
pnamereturn

,0
Joshua Bae,[elec after dinner?]
Kevin Tu,"[elec after dinner?: 1., daddy orchard is disappointed in me., hurt me more daddy orchard., 3 slices boi., disappointment overflow (volunteer)., disappointment overflow (host)., rice dropout.]"
Michael Sprintson,"[after dinner?, disappointment overflow., gtx 970 (3.5/4).]"
Kevin Mejia,[Daddy Orchard's Disappointment.]
Armando Amigon,"[post disappointment sex nachos., Disappointment Overflow.]"
Zach Alvear,[monogamous post disappointment sex nachos.]
Anirudh Kuchibhatla,[monogamous post disappointment sex nachos overflow.]
Patrick Chickey,"[M.P.D.S.O., M.P.D.S.N.O., professional banner.]"


In [20]:
plot_dict(ptimereturn.to_dict())
plt.savefig(f'pictures/nickname_history_{participant}.png')
plt.close()

 ## group name changes

In [21]:
group_name_changes_timeline(total_messages)

{'Shockj Bojs.': 10316133189,
 'jllumjnatj.': 7929928712,
 'Shocki Bois.': 1950046060,
 'every night at fredys.': 1914519071,
 'sad boi elecs (in quarantine).': 1217456814,
 'The Alpha ELEC Group Chat.': 1045727243,
 'sad boi elecs.': 588028079,
 'Opensource HW Answers.': 580061751,
 'Funded By Simar (Satire).': 525119715,
 'wwj troops (ð\x9f¤¬).': 356026100,
 '220 Volunteers (Satire).': 174060342,
 'misery loves Simar.': 68213759,
 '220 Volunteers.': 12541105,
 'ELEC Goon Squad.': 3622140,
 'misery loves company.': 282359,
 "DoJo's Dojo.": 82395,
 'ww4 troops (angry).': 65235,
 'Armando Is A Bully.': 61724,
 'wwj troops (angry).': 49476,
 'A N G E R J  Bojs.': 44964,
 'Fredy and the Bois.': 37049,
 'Fredy and the Oppressed.': 33183,
 "Fredy's Tyranny.": 27808,
 'wwj troops (angery).': 24691,
 "Fredy's and the Bois.": 11259,
 'all nights at fredys.': 1587}

### usage per person

In [22]:
usage_participant_highlighted(total_messages, participant_names, participant)
plt.close()

In [23]:
word_usage_highlighted(total_messages, participant_names, 'code')
plt.close()

### photos count

In [24]:
most_common_photo_sender(total_messages)

{'Michael Sprintson': 331,
 'Kevin Tu': 316,
 'Kevin Mejia': 314,
 'Armando Amigon': 265,
 'Justin Cheung': 218,
 'Feme Longe': 165,
 'Rene Carballo': 146,
 'Joshua Bae': 135,
 'Anirudh Kuchibhatla': 117,
 'Patrick Chickey': 98,
 'Zach Alvear': 71,
 'Fredy Martinez': 65}

### reading level

In [25]:
# rdng_levels = {}
# for pn in participant_names:
#     pmes = [i['content'] for i in total_messages if 'content' in i and pn == i['sender_name']]
#     textblob = reduce(lambda a,b: a + ' ' + b, pmes)
#     rdng_levels[pn] = textstat.smog_index(textblob)

In [26]:
# pd.Series(rdng_levels).sort_values(ascending=False)

In [27]:
# import textstat

In [28]:
# textstat.gunning_fog('Esteemed friends and colleagues, please disregard this foolish momentary abandonment of my eloquence related endeavors. When faced with the visage of statistics and scholarship, feeling the exhilarating thrill of problem solving, I briefly forgot myself and the notions to which I had previously committed myself. As I mentioned before, the further we become removed from this discussion the more likely and frequent such lapses will become, but such is life.')

### most common tags per person

In [29]:
tag_correlation_matrix(participant_names, total_messages)
plt.close()

Let's find who is most likely to reply to who - should use hidden markov model

In [30]:
gen_reply_matrix(total_messages_with_id, participant_names, False)
plt.close()